### QC+ for Cai 2020 dataset

#### Objective: Run advanced QC for the Cai 2020 dataset, including data normalization, log transformation, and UMAP visualization


**Developed by**: Mairi McClean

**Affiliation**: Institute of Computational Biology - Computational Health Centre - Helmholtz Munich

**v230207**


### Load modules

In [ ]:
import anndata
import logging
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sb
import matplotlib.pyplot as plt
import igraph as ig
from matplotlib import colors
from matplotlib import rcParams

: 

#### Log file and figure output settings

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 160, color_map = 'RdPu', dpi_save = 180, vector_friendly = True, format = 'svg')

### Read in anndata object

In [ ]:
adata = sc.read_h5ad('/Users/mairi.mcclean/github/data/tb_pbmc_datasets/qcd_objects/2111_2511_exported_objects/23/Nathan2021_PBMC_TB_QCed_pre-process_mm221123.h5ad')
adata

### Data normalization

target_sum taken from Scanpy tutorial [https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html]

If exclude_highly_expressed=True, very highly expressed genes are excluded from the computation of the normalization factor (size factor) for each cell. 
> This is meaningful as these can strongly influence the resulting normalized values for all other genes [Weinreb17]."

1e4 changed to 1e6; option to exclude highly expressed genes, set as true - CHANGE THIS FIRST before changing any other variable to observe effects on PCA output


In [ ]:
sc.pp.normalize_total(adata, target_sum=1e6, exclude_highly_expressed=True)

### Data log transformation

In [ ]:
sc.pp.log1p(adata)

### Identify highly variable genes

Code from Carlos' notebook scVI_exploratory_analysis; Seurat should be used for all generative models
Number of genes (top_genes) based on how mixed the sample is, and computational power. Highest gene number is 10,000
Batch_key is related to the data that we want to perform the filtering on
Carlos runs between 4000 (low RAM) and 7000
subset = True will remove all non-variable genes

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 8000,
    layer = "counts",
    batch_key = "sample",
    subset = True
)

In [ ]:
adata.var.head()


In [ ]:
sc.pl.highly_variable_genes(adata)


We want to see between 0.25 and 0.75 after 1.
This particular pattern either means that the data is garbage or that it is highly significant, caused by disease.
Could see if changing the number of genes from between 4000 to 7000 affects it

### PCA

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
adata

Can add covariates here below; can remove frames from image

In [ ]:
adata.obs.head()

In [ ]:
sc.pl.pca(adata, color = ['sample', 'donor', 'n_genes_by_counts', 'status'], wspace=0.5)

### Computing neighbourhood graph

Carlos chooses 50 neighbours, and 50 PCs (PCs used to be taken from elbow graph)

This step is done to create a 'scaffold' of the data; UMAP embedding will then place data over scaffold to see how it fits

In [ ]:
sc.pp.neighbors(adata, n_neighbors=50, n_pcs=50)



### UMAP embedding

#### Clustering

In [ ]:
# added to avoid error arising from running subsequent cell on its own

sc.tl.leiden(adata)

In [ ]:
sc.tl.umap(adata)


In [ ]:
adata.var.head()


In [ ]:
adata.obs.head()

In [ ]:
sc.pl.umap(adata, color=[   
    'leiden',  
    'percent_chrY', 
    'XIST-counts',  
    'pct_counts_ribo',  
    'percent_mt2', 
    'n_genes_by_counts', 
    'n_counts',   
    'predicted_doublets',   
    'sample',], size = 1, wspace=0.50)

In [ ]:
# Each sample has made it's own cluster, which is batch effect
# choose variety of covariates from obs for panel

In [ ]:
# Now we can use the following code to plot the scaled and corrected gene expression data

sc.pl.umap(adata, use_raw=False, )

### Writing out object


In [ ]:
adata.write('/Users/mairi.mcclean/github/data/tb_pbmc_datasets/qc_plus_visualisation/230207_Nathan2021_MM_QCplus.h5ad')
# needs extension .h5ad